To install Pyomo:  conda install -c conda-forge pyomo 

To install glpk solver:  conda install -c conda-forge glpk

In [2]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory
import pandas as pd
import os
import numpy as np
import time
from os import listdir
from os.path import isfile, join

In [3]:
#Additional solver imports for Pyomo
from gurobipy import *
import gurobipy as gp
from gurobipy import GRB
import mosek
import xpress
from pyomo.core.expr.current import identify_variables

In [4]:
def createDataForL1FromDataFrame(df):
    explanatoryTraining = []
    nObservations = len(df['Response'])
    nExplanatorys = len(df.columns)-1
    responseTraining = df['Response'].tolist()
    for i in range(1,len(df.columns)):
        explanatoryTraining.append(df[f'Explanatory{i}'].tolist())
            
    return explanatoryTraining, responseTraining, nObservations, nExplanatorys

In [5]:
def inertTestDataSeries(dfOriginal,dfSeriesToTest,betaValues,dualVariablesFitted, df_BasisX1_inv):
    #df contains a set of points each of which will be inert tested and a list containing inertness of each point as True or False will be returned.
    #dualVariablesFitted=[dualVariableValuesOriginal[i] for i in indexesOfFittedPointsOriginal]
    #inertTestListReturn=[]
    
    tempResponse=dfSeriesToTest[0].copy()
    dfSeriesToTest[0]=1
    
    pointsDistanceToL1Hyperplane = tempResponse-np.sum(np.matmul(dfSeriesToTest,betaValues))
    pointsPositionToL1Hyperplane = np.sign(pointsDistanceToL1Hyperplane)
    #pointsPositionToL1Hyperplane = np.sum(df_BasisX1_inv[len]
    #print('len(dualVariablesFitted): ', len(dualVariablesFitted),'len(dfSeriesToTest): ', len(dfSeriesToTest),'len(df_BasisX1_inv): ', len(df_BasisX1_inv), )
    inertnessValue = dualVariablesFitted-pointsPositionToL1Hyperplane*np.matmul(np.asarray(dfSeriesToTest),np.asarray(df_BasisX1_inv))
    
    #This check process can be done faster
    inertnessOfaPoint = inertnessValue[(inertnessValue>=-1) & (inertnessValue<=1)]
    pointIsInert = len(inertnessOfaPoint)==len(inertnessValue)
    #inertTestListReturn.append(pointIsInert)
    
    return pointIsInert, abs(pointsDistanceToL1Hyperplane), inertnessValue, np.sign(pointsDistanceToL1Hyperplane)

In [6]:
# Updated on 8/26/22 pyo.VarList is tested. Indexing starts at 1, this is incorparated to the code in this cell
# L1 REGRESSION INITIAL SOLVING FUNCTION
def L1RegressionModelNormalStartPyomo (explanatoryTraining,responseTraining,manualBetaValues): #This function has two if statement in it, it should not be used for ultimate speed comparison
    #Below different options for data creation are created depending on the parameters passed with L1RegressionModel.
    functionStartTime=time.perf_counter()
    
    nObservations=len(responseTraining)
    nExplanatorys=len(explanatoryTraining) 
#--------------------------------------------------------------------------------------------------------------------------        
    model = pyo.ConcreteModel()
    model.betaValues = pyo.VarList()
    for i in range(len(explanatoryTraining)+1):
        model.betaValues.add()
    #model.betaValues = pyo.Var(range(len(explanatoryTraining)+1))
    #model.zPlus = pyo.Var(range(len(explanatoryTraining[0])), domain=pyo.NonNegativeReals)
    #model.zMinus = pyo.Var(range(len(explanatoryTraining[0])), domain=pyo.NonNegativeReals)
    model.zPlus = pyo.VarList(domain=pyo.NonNegativeReals)
    for i in range(len(explanatoryTraining[0])):
        model.zPlus.add()
    model.zMinus = pyo.VarList(domain=pyo.NonNegativeReals)
    for i in range(len(explanatoryTraining[0])):
        model.zMinus.add()


    #Objective function
    temp_expr = 0
    for i in range(nObservations):               
        temp_expr+=(model.zPlus[i+1]+model.zMinus[i+1])
    model.obj = pyo.Objective(expr = temp_expr, sense=pyo.minimize)
    
    #Constraints
    model.ConstraintSet=pyo.ConstraintList()
    for i in range(nObservations):
        #Babayaga
        #for j in range(len(explanatoryTraining)):
        #    print('i: ', i, '\n type(model.betaValues[1])', type(model.betaValues[1]), '\n type(model.zPlus[i+1])',\
        #          type(model.zPlus[i+1]), '\n type(model.zMinus[i+1])', type(model.zMinus[i+1]),\
        #          '\n type(model.betaValues[j+2])', type(model.betaValues[j+2]),\
        #          '\n type(explanatoryTraining[j][i])', type(explanatoryTraining[j][i]), explanatoryTraining[j][i],\
        #          '\n type(len(explanatoryTraining))', type(len(explanatoryTraining)),\
        #         '\n type(responseTraining[i])', type(responseTraining[i]))
        model.ConstraintSet.add(model.betaValues[1]+model.zPlus[i+1]-model.zMinus[i+1]+ \
    sum(model.betaValues[j+2]*explanatoryTraining[j][i] for j in range(len(explanatoryTraining)))==responseTraining[i])
    #def constraint_rule(m, i):
    #    return m.betaValues[0]+m.zPlus[i+1]-m.zMinus[i+1]+ \
    #sum(m.betaValues[j+1]*explanatoryTraining[j][i] for j in range(len(explanatoryTraining)))==responseTraining[i]
    #Adds sets of constraints having indices: range(len(explanatoryTraining[0])) by calling the constraint rule above.
    #model.ConstraintSet = pyo.Constraint(range(len(explanatoryTraining[0])), rule=constraint_rule)
    
    #Manuel beta values
    if len(manualBetaValues) != 0:
        def manual_constraint_rule(m, i):
            return m.betaValues[i] == manualBetaValues[i]
        model.ManualConstraintSet = pyo.Constraint(range(len(explanatoryTraining)+1), rule=manual_constraint_rule)
    
    model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)
    opt = pyo.SolverFactory('glpk')
    #model.varList = pyo.VarList()
    #opt = pyo.SolverFactory('scip', executable="./scip")
    result=opt.solve(model)
    
    functionEndTime=time.perf_counter()
    functionRunTime=functionEndTime-functionStartTime
    return model, functionRunTime,result

In [7]:
# Updated on 8/26/22 pyo.VarList is tested. 
# Indexing starts at 1, this is incorparated to the code in this cell.
# Python related indexing will start with zero but pyomo relating indexing will start with 1.
# This cell reconciles the discrepency mentioned above.

def streamingL1RegressionColdStartPyomo(dfOriginal):
    #This function takes the historical data set and run L1 regression iteratively on it starting with the first m x m matrix exert of the original data set.
    #Start function timer
    print('streamingL1RegressionColdStartPyomo is initiated')
    functionStartTime=time.perf_counter()
    
    #Initialize variables
    manualBetaValues=list()
    totalSolverRunTime=0
    totalIterationCount=0    
    
    #Describe parameters
    lenOriginal=dfOriginal.shape[0]
    dimensionL1=dfOriginal.shape[1]
    print('lenOriginal: ', lenOriginal, 'dimensionL1: ', dimensionL1)
    totalNumberOfPointsToAdd=lenOriginal-dimensionL1+1
    
    #Partition the original data set into m by m and n-m by m sets. 
    dfStreaming = dfOriginal.iloc[0:dimensionL1].copy()
    dfNotStreamed = dfOriginal.iloc[dimensionL1:lenOriginal].copy()
    
    
    #----Solve L1 regression on the initial data set which is m by m--------
    explanatoryTraining, responseTraining, nObservations, nExplanatorys = createDataForL1FromDataFrame(dfStreaming)
    
    modelX, functionRunTimeNormal, results=L1RegressionModelNormalStartPyomo (explanatoryTraining,responseTraining,manualBetaValues)
    
    functionStartTime=time.perf_counter()
    instance = modelX.create_instance()

    #----Start iterating on the rest of the data set one by one simulating streaming data------
    for index, row in dfNotStreamed.iterrows():
        
        #Add a row to data frame from back. 
        dfStreaming.loc[len(dfStreaming)]=row.copy()
        
        #Create a new variable in zPlus and zMinus variables
        #print('number of points: ', index+1)
        #print('len(zPlus): ',len(modelX.zPlus))
        #new_v = modelX.varList.add()
        instance.zPlus.add()
        instance.zMinus.add()


        #Create the new set of constraints for L1Model
        temp_sum = 0
        temp_sum += instance.betaValues[1]
        for j in range(dimensionL1-1):
            temp_sum += instance.betaValues[j+2]*row[j+1]
        temp_sum += instance.zPlus[index+1]-instance.zMinus[index+1]
        instance.ConstraintSet.add(temp_sum == row[0])

        #Update the objective
        instance.obj.expr += (instance.zPlus[index+1] + instance.zMinus[index+1])

        #Solve the updated model - warmstart toogle is off
        opt = pyo.SolverFactory('glpk')
        results = opt.solve(instance)
        #opt.solve(instance)
        #modelX.display()
        #print(results)
        if index%500==0:
            print(f'{index}th point processed in streamingL1RegressionColdStartPyomo')

    


    #print('Beta0: ', pyo.value(instance.betaValues[1]), 'Beta1: ', pyo.value(instance.betaValues[2]),\
    #      'Beta2: ', pyo.value(instance.betaValues[3]), 'Beta3: ', pyo.value(instance.betaValues[4]))            
            
    #print all ("Duals") 
    #for c in instance.component_objects(pyo.Constraint, active=True):
    #    print ("   Constraint",c)
    #    for index in c:
    #        print ("      ", index, instance.dual[c[index]])
        
    #Print all variables
    #for v in instance.component_objects(pyo.Var, active=True):
    #    print("Variable",v)  
    #    for index in v:
    #        print ("   ",index, pyo.value(v[index]))
    
    #varsConstrains = identify_variables(instance.ConstraintSet)
    #print('varsConstrains: ', varsConstrains)
    functionEndTime=time.perf_counter()
    functionRunTime=functionEndTime-functionStartTime
    functionReturn={'functionRunTime': functionRunTime}
    return functionReturn , results

In [8]:
# 08/30/22

def streamingL1RegressionColdInertPyomo(dfOriginal):
    #This function takes the historical data set and run L1 regression iteratively on it starting with the first m x m matrix exert of the original data set.
    #Start function timer
    print('streamingL1RegressionColdInertPyomo is initiated')
    functionStartTime=time.perf_counter()
    
    #Initialize variables
    manualBetaValues=list()
    totalSolverRunTime=0
    totalIterationCount=0
    notInertPointIndexes=[]
    differentialListWhenArrivalIsNotInert=[]
    differentialList=[]
    inertRunList=[]
    
    #Describe parameters
    lenOriginal=dfOriginal.shape[0]
    dimensionL1=dfOriginal.shape[1]
    nDimensionRange= range(dimensionL1)
    print('lenOriginal: ', lenOriginal, 'dimensionL1: ', dimensionL1)
    totalNumberOfPointsToAdd=lenOriginal-dimensionL1+1
    
    #Partition the original data set into m by m and n-m by m sets. 
    dfStreaming = dfOriginal.iloc[0:dimensionL1].copy()
    dfNotStreamed = dfOriginal.iloc[dimensionL1:lenOriginal].copy()
    
    
    #----Solve L1 regression on the initial data set which is m by m--------
    explanatoryTraining, responseTraining, nObservations, nExplanatorys = createDataForL1FromDataFrame(dfStreaming)
    
    modelX, functionRunTimeNormal, results=L1RegressionModelNormalStartPyomo (explanatoryTraining,responseTraining,manualBetaValues)
    
    differential=0
    differentialList.append(differential)
    
    functionStartTime=time.perf_counter()
    instance = modelX.create_instance()
    
    #Get the indexes of the fitted points
    indexesOfFittedPoints=[]
    for i in np.arange(nObservations):
        if (pyo.value(instance.zPlus[i+1]) + pyo.value(instance.zMinus[i+1])) == 0:
            indexesOfFittedPoints.append(i)
    #print('len(indexesOfFittedPoints)1: ', len(indexesOfFittedPoints))
    #Get the L1 regression hyperplane parameters - beta values 
    betaValues=[]
    for i in nDimensionRange:
        betaValues.append(pyo.value(instance.betaValues[i+1]))
        
        
        
    #get basisX1 inverse    
    dfBasisX1=dfOriginal.iloc[indexesOfFittedPoints].copy()
    dfBasisX1['Response']=1
    df_BasisX1_inv = pd.DataFrame(np.linalg.pinv(dfBasisX1.values), dfBasisX1.columns, dfBasisX1.index)
    
    # Get the dual variable values corresponding to the fitted points
    dualVariableValuesFittedPoints=[]
    for c in instance.component_objects(pyo.Constraint, active=True):
        for index in indexesOfFittedPoints:
            dualVariableValuesFittedPoints.append(instance.dual[c[index+1]])
    
   
    #----Start iterating on the rest of the data set one by one simulating streaming data------
    #You can save some time using index instead of counter
    inertRunAmount=0
    #----Start iterating on the rest of the data set one by one simulating streaming data------
    for index, row in dfNotStreamed.iterrows():
        if index%500==0:
            print(f'{index}th point processed in streamingL1RegressionColdInertPyomo')
        #Add a row to data frame from back. 
        dfStreaming.loc[len(dfStreaming)]=row.copy()
        
        instance.zPlus.add()
        instance.zMinus.add()


        #Create the new set of constraints for L1Model
        temp_sum = 0
        temp_sum += instance.betaValues[1]
        for j in range(dimensionL1-1):
            temp_sum += instance.betaValues[j+2]*row[j+1]
        temp_sum += instance.zPlus[index+1]-instance.zMinus[index+1]
        instance.ConstraintSet.add(temp_sum == row[0])

        #Update the objective
        instance.obj.expr += (instance.zPlus[index+1] + instance.zMinus[index+1])
        

        inertResult, estimationErrorForTheNewPoint, dualVariableValuesFittedPoints, positionOfArrival = \
        inertTestDataSeries(dfStreaming,row,betaValues,dualVariableValuesFittedPoints, df_BasisX1_inv)
        

        if inertResult == False:
            
            nObservations=dfStreaming.shape[0]
            inertRunList.append(inertRunAmount)
            inertRunAmount=0
            notInertPointIndexes.append(index)
            
            opt = pyo.SolverFactory('glpk')
            results = opt.solve(instance)    
            
            indexesOfFittedPoints=[]
            indexesOfUnfittedPoints=[]
            #Get the indexes of the fitted points and unfitted points
            for i in np.arange(nObservations):
                if (pyo.value(instance.zPlus[i+1]) + pyo.value(instance.zMinus[i+1])) == 0:
                    indexesOfFittedPoints.append(i)
                else:
                    indexesOfUnfittedPoints.append(i)
            #print('len(indexesOfFittedPoints)2: ', len(indexesOfFittedPoints))
            #print('indexesOfFittedPoints: ', indexesOfFittedPoints, 'indexesOfUnfittedPoints: ', indexesOfUnfittedPoints)
                
            # Get the dual variable values with the following code
            #Find the differential by summing up the dual variable values corresponding the unfitted point indexes.
            # c[index]: is the name of the indexth constraint
            # modelX.dual[<constraint name>]: gets the dual variable value for the indexth constraint
            # c: is the parent name for the constraints set. It is also a list containing other constraint names and may be more.
            # c is an interesting object, it is a list containing constraint names, it is also a range list of the indexes of the constraints.
            dualVariableValuesFittedPoints=[]
            differential=0
            for c in instance.component_objects(pyo.Constraint, active=True):
                for index in c:
                    if index-1 in indexesOfFittedPoints:
                        dualVariableValuesFittedPoints.append(instance.dual[c[index]])
                    else:
                        differential+=instance.dual[c[index]]
            #print('len(indexesOfFittedPoints)3: ', len(indexesOfFittedPoints))
            #print('indexesOfFittedPoints: ', indexesOfFittedPoints)
            differentialListWhenArrivalIsNotInert.append(differential)
            differentialList.append(differential)
            #print('index: ', index, 'differential: ', differential, 'positionOfArrival: ', positionOfArrival, 'inert? : ', inertResult)

            #Get the L1 regression hyperplane parameters - beta values 
            betaValues=[]
            for i in nDimensionRange:
                betaValues.append(pyo.value(instance.betaValues[i+1]))            
                
            
            #Find the inverse of the basisX1
            dfBasisX1=dfStreaming.iloc[indexesOfFittedPoints].copy()
            dfBasisX1['Response']=1
            df_BasisX1_inv =np.linalg.pinv(dfBasisX1)
            

        else:
            #The arrival is inert, update the relevant lists.
            inertRunAmount+=1
            differential+=positionOfArrival
            differentialList.append(differential)
            #print('index: ', index, 'differential: ', differential, 'positionOfArrival: ', positionOfArrival, 'inert? : ', inertResult)            
    

    inertRunList.append(inertRunAmount)        
    functionEndTime=time.perf_counter()
    functionRunTime=functionEndTime-functionStartTime
    inertPointPercentage = (len(dfNotStreamed)-len(notInertPointIndexes))/len(dfNotStreamed)
    functionReturn={'functionRunTime': functionRunTime, 'notInertPointIndexes': notInertPointIndexes,\
                    'inertPointPercentage': inertPointPercentage, 'differentialList': differentialList,\
                   'differentialListWhenArrivalIsNotInert': differentialListWhenArrivalIsNotInert,\
                   'inertRunList': inertRunList}
    return functionReturn , results

# ==========================

In [ ]:
#Put the example data in the same folder as this .ipynb file.
#The example data is: RealEstateValuationData.xlsx
#APPLY EXPERIMENTS ON A DATA SET
#Babayaga
experimentStartTime=time.perf_counter()

listColdStartTimeFunction=[]
listColdInertTimeFunction=[]
listColdvsColdInertFunction=[]
inertPointPercentageList=[]
nonInertPointIndexList=[]
nList=[]
mList=[]




#print('newPath: ', newPath)
dataName='RealEstateValuationData.xlsx'
dfOriginal=pd.read_excel(dataName)
#Test this shuffeling method (resets the point indexes)
#dfOriginal = dfOriginal.sample(frac=1).reset_index(drop=True)
#As well as this
#dfOriginal = dfOriginal.sample(frac=1)
#number of points
n=np.shape(dfOriginal)[0]
#dimension
m=np.shape(dfOriginal)[1]
#
nList.append(n)
mList.append(m)

#Create a string parameter --> SpeedTest + <file name>
#Do speed tests

print('naive online L1 regression is started')
resultStreamingColdStart, resultsColdStart = streamingL1RegressionColdStartPyomo(dfOriginal)
print('naive online L1 regression is done')
resultStreamingColdInert, resultsColdInert= streamingL1RegressionColdInertPyomo(dfOriginal)
print('online L1 regression with inertness test is done')

listColdStartTimeFunction.append(resultStreamingColdStart['functionRunTime'])
listColdInertTimeFunction.append(resultStreamingColdInert['functionRunTime'])
listColdvsColdInertFunction.append((resultStreamingColdStart['functionRunTime']-resultStreamingColdInert['functionRunTime'])/resultStreamingColdStart['functionRunTime'])
inertPointPercentageList.append(resultStreamingColdInert['inertPointPercentage'])
nonInertPointIndexList.append(resultStreamingColdInert['notInertPointIndexes'])

        
dfToAppend = pd.DataFrame ({'data names': 'WaterResourcesDataColumn1Excluded.csv',\
                            'cold start run (seconds) function time':listColdStartTimeFunction,\
                            'cold start + inert zones run (seconds) function time':listColdInertTimeFunction,\
                            '(cold start) vs (cold start + inert zones) (%100) function time':listColdvsColdInertFunction,\
                            'Inert point percent': inertPointPercentageList,\
                            'nonInertPointIndexList': nonInertPointIndexList,\
                            'n': nList,\
                            'm': mList
                           }
                          )

#dfToAppend = pd.DataFrame(dictionaryToAppend)
writer = pd.ExcelWriter(f'ExperimentResults{dataName}.xlsx' , engine='xlsxwriter')
dfToAppend.to_excel(writer, index=True)
writer.save()

naive online L1 regression is started
streamingL1RegressionColdStartPyomo is initiated
lenOriginal:  414 dimensionL1:  7
